In [41]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from datetime import datetime
import wandb
import argparse
from torch import nn, optim
from IPython.display import Image

# TitanicDataset

In [42]:
class TitanicDataset(Dataset):
    def __init__(self, X, y):
        '''데이터셋 초기화'''
        self.X = torch.FloatTensor(X)
        self.y = torch.LongTensor(y)
        
    def __len__(self):
        '''데이터셋 전체 길이를 반환'''
        return len(self.X)
    
    def __getitem__(self, idx):
        '''데이터셋의 특정 인덱스(feature) 반환'''
        feature = self.X[idx]
        target = self.y[idx]    # 타겟 데이터
        return {'input':feature, 'target': target}
    
    def __str__(self):
        '''데이터셋의 정보 알려주는 함수'''
        str = "Data Size: {0}, Input Shape: {1}, Target Shape: {2}".format(len(self.X), self.X.shape, self.y.shape
        )
        return str

# TitanicTestDataset

In [43]:
class TitanicTestDataset(Dataset):
  def __init__(self, X):
    '''데이터셋 초기화'''
    self.X = torch.FloatTensor(X)

  def __len__(self):
    '''데이터셋 전체 길이를 반환'''
    return len(self.X)

  def __getitem__(self, idx):
    '''데이터셋의 특정 인덱스(feature) 반환'''
    feature = self.X[idx]
    return {'input': feature}

  def __str__(self):
    '''데이터셋의 정보 알려주는 함수'''
    str = "Data Size: {0}, Input Shape: {1}".format(
      len(self.X), self.X.shape
    )
    return str

# get_preprocessed_dataset

In [44]:
def get_preprocessed_dataset():
    # 파일의 절대 경로 설정
    CURRENT_FILE_PATH = os.path.dirname(os.path.abspath("__file__"))
    
    # train.csv 파일 경로 설정
    train_data_path = os.path.join(CURRENT_FILE_PATH, "train.csv")
    # test.csv 파일 경로 설정
    test_data_path = os.path.join(CURRENT_FILE_PATH, "test.csv")
    
    train_df = pd.read_csv(train_data_path)
    test_df = pd.read_csv(test_data_path)
    
    # train.csv : PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Enbarked
    # test.csv : PassengerId, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Enbarked
    # 데이터프레임을 연결할 때 컬럼의 순서나 이름이 일치하지 않아도, 해당 컬럼 이름이 존재하는 경우 데이터를 합침
    all_df = pd.concat([train_df, test_df], sort=False)

    all_df = get_preprocessed_dataset_1(all_df)

    all_df = get_preprocessed_dataset_2(all_df)

    all_df = get_preprocessed_dataset_3(all_df)

    all_df = get_preprocessed_dataset_4(all_df)

    all_df = get_preprocessed_dataset_5(all_df)

    all_df = get_preprocessed_dataset_6(all_df)

    train_X = all_df[~all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)
    train_y = train_df["Survived"]

    test_X = all_df[all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)

    dataset = TitanicDataset(train_X.values, train_y.values)
    #print(dataset)
    train_dataset, validation_dataset = random_split(dataset, [0.8, 0.2])
    test_dataset = TitanicTestDataset(test_X.values)
    #print(test_dataset)

    return train_dataset, validation_dataset, test_dataset

# get_preprocessed_dataset_1

In [45]:
def get_preprocessed_dataset_1(all_df):
    '''Pclass별 Fare 평귱값을 사용하여 Fare 결측치 메우기'''
    # Pcalss 와 Fare을 선택하여 데이터 추출 후 Pclass로 그룹화
    # 그 후 나온 평균 값을 Fare_mean에 저장
    Fare_mean = all_df[["Pclass", "Fare"]].groupby("Pclass").mean().reset_index()
    # Fare_mean 열의 이름을 Pclass 와 Fare_mean으로 변경
    Fare_mean.columns = ["Pclass", "Fare_mean"]
    # Fare_mean 데이터프레임을 Pclass 열을 기준으로 left join 하여 병합
    # Pclass 열을 기준으로 Fare의 결측치를 Fare_mean 값으로 채울 수 있음
    all_df = pd.merge(all_df, Fare_mean, on="Pclass", how="left")
    all_df.loc[(all_df["Fare"].isnull()), "Fare"] = all_df["Fare_mean"]
    
    return all_df

# get_preprocessed_dataset_2

In [46]:
def get_preprocessed_dataset_2(all_df):
    '''name을 세 개의 컬럼으로 분리하여 다시 all_df에 합침'''
    # , 또는 .을 기준으로 이름을 세 부분으로 분리 ex) Mr.Owen Karris -> Mr 과 Owen 과 Karris로 분리
    name_df = all_df["Name"].str.split("[,.]", n=2, expand=True)
    # "family_name," "honorific," "name" 순서대로 이름이 분리
    name_df.columns = ["family_name", "honorific", "name"]
    # 각 이름들을 공백을 제거한 형태로 저장
    name_df["family_name"] = name_df["family_name"].str.strip()
    name_df["honorific"] = name_df["honorific"].str.strip()
    name_df["name"] = name_df["name"].str.strip()
    # name_df을 열 방향(axis=1)으로 합침
    all_df = pd.concat([all_df, name_df], axis=1)

    return all_df

# get_preprocessed_dataset_3

In [47]:
def get_preprocessed_dataset_3(all_df):
    ''' hoorific별 Age 평균값을 사용하여 Age 결측치 메우기'''
    # honorific열 과 Age열을 honorific으로 묶고 중앙을 계산하여 Age값에서 반올림한다.
    honorific_age_mean = all_df[["honorific", "Age"]].groupby("honorific").median().round().reset_index()
    honorific_age_mean.columns = ["honorific", "honorific_age_mean", ]
    # honorific_age_mean 데이터프레임을 honorific 열을 기준으로 left join 하여 병합
    # honorific 열을 기준으로 Age 열의 결측치를 honorific_age_mean 값으로 채울 수 있음
    all_df = pd.merge(all_df, honorific_age_mean, on="honorific", how="left")
    all_df.loc[(all_df["Age"].isnull()), "Age"] = all_df["honorific_age_mean"]
    # 필요 없어진 honorific_age_mean 열을 삭제
    all_df = all_df.drop(["honorific_age_mean"], axis=1)

    return all_df

# get_preprocessed_dataset_4

In [48]:
def get_preprocessed_dataset_4(all_df):
    # Parch열 과 SibSp열을 합쳐, 가족수(family_num) 컬럼 새롭게 추가
    # Parch : 부모/자녀 수
    # SibSp : 형제/자매/배우자 수
    all_df["family_num"] = all_df["Parch"] + all_df["SibSp"]
    
    # 혼자탑습(alone) 컬럼 새롭게 추가
    all_df.loc[all_df["family_num"] == 0, "alone"] = 1
    all_df["alone"].fillna(0, inplace=True)
    
    # 학습에 불필요한 컬럼 제거
    all_df = all_df.drop(["PassengerId", "Name", "family_name", "name", "Ticket", "Cabin"], axis=1)
    
    return all_df

# get_preprocessed_dataset_5

In [49]:
def get_preprocessed_dataset_5(all_df):
    ''' honorific 값 개수 줄이기'''
    # honorific열에서 Mr, Miss, Mrs, Master이 아닌 경우 해당 값을 other로 변경
    all_df.loc[
    ~(
            (all_df["honorific"] == "Mr") |
            (all_df["honorific"] == "Miss") |
            (all_df["honorific"] == "Mrs") |
            (all_df["honorific"] == "Master")
    ),
    "honorific"
    ] = "other"
    # fillna 함수를 사용하여 탑승지(Embarked)열의 결측치를 missing으로 변경
    # inplace=False가 기본, True로 설정하면 데이터프레임 자체를 변경
    all_df["Embarked"].fillna("missing", inplace=True)

    return all_df

# get_preprocessed_dataset_6

In [50]:
def get_preprocessed_dataset_6(all_df):
    '''카테고리 변수(범주형, 문자열)를 LabelEncoder를 사용하여 수치값(정수)으로 변경하기'''
    # all_df에서 데이터 타입이 문자열(object)인 모든 열을 찾아서 category_features에 저장
    category_features = all_df.columns[all_df.dtypes == "object"]
    # LabelEncoder는 범주형 데이터를 정수로 변환하는 데 사용되는 클래스
    from sklearn.preprocessing import LabelEncoder
    for category_feature in category_features:
        le = LabelEncoder()
        if all_df[category_feature].dtypes == "object":
            # fit 메서드를 사용하여 해당 열의 고유한 범주값들을 학습
            le = le.fit(all_df[category_feature])
            # transform 메서드를 사용하여 해당 열의 모든 값을 해당 범주의 정수로 변환
            all_df[category_feature] = le.transform(all_df[category_feature])
            
    return all_df

# 신경망 정의

In [51]:
from torch import nn
class MyModel(nn.Module):
    '''신경망 정의 / MyModel클래스는 nn.Module클래스를 상속받아 pytorch의 모델 기능을 활용할 수 있도록 함'''
    # input : feature의 수
    # n_output : 출력되어 나오는 결과
    def __init__(self, n_input, n_output):
        # 부모 클래스의 nn.Module의 생성자를 호출
        super().__init__()
        
        # 시퀀셜(Sequential) 모델을 사용, 이를 통해 여러 계층을 순차적으로 쌓을 수 있음
        # 시퀀셜 모델은 각 계층을 순차적으로 연결
        self.model = nn.Sequential(
            # n_input에서 30개의 뉴런을 가진 완전 연결 레이어를 생성
            nn.Linear(n_input, 30),
            nn.LeakyReLU(0.1),
            nn.Linear(30, 30),
            nn.LeakyReLU(0.1),
            nn.Linear(30, n_output),
            # 30개의 뉴런에서 출력 클래스 수 n_output으로 연결
            #nn.Linear(30, n_output),
        )
        
    # foward
    #  모델에 입력 데이터 x를 전달하여 출력을 반환
    def forward(self, x):
        '''foward는 Sequential을 묶은 걸 self.model에 넣어 줬기 때문에 x = self.model(x)를 하면 fpward기 쭉 올라감'''
        # 입력 데이터 x를 모델의 시퀀셜 레이어에 전달하여 feed fowarding 수행
        x = self.model(x)
        return x

# test

In [52]:
def test(test_data_loader):
    '''모델을 테스트 하는 부분'''
    print("[TEST]")
    # iter를 사용하여 감싸준 다음 next를 사용하여 첫번째 미니배치를 배치로 넘김
    batch = next(iter(test_data_loader))
    print("{0}".format(batch['input'].shape))
    my_model = MyModel(n_input=11, n_output=2)
    output_batch = my_model(batch['input'])
    # 모델의 출력 중에서 각 샘플에 대한 가장 높은 값을 가진 클래스의 인덱스를 선택하여 예측
    prediction_batch = torch.argmax(output_batch, dim=1)
    # 인덱스를 892부터 시작하도록 설정, 첫번째 테스트 데이터의 인덱스 번째
    for idx, prediction in enumerate(prediction_batch, start=892):
        print(idx, prediction.item())

# main문

In [53]:
if __name__ == "__main__":
    # wandb 초기화
    wandb.init(project="titanic", name="LeakyReLU")
    
    train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()

    print("train_dataset: {0}, validation_dataset.shape: {1}, test_dataset: {2}".format(
        len(train_dataset), len(validation_dataset), len(test_dataset)
    ))
    print("#" * 50, 1)

    for idx, sample in enumerate(train_dataset):
        print("{0} - {1}: {2}".format(idx, sample['input'], sample['target']))

    print("#" * 50, 2)

    train_data_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
    validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=16, shuffle=True)
    # test_dataset 전체를 배치로 받아도 되는 이유 : 다른 데이터에 비해 데이터의 크기가 작기 때문
    test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

    # 모델 정의
    my_model = MyModel(n_input=11, n_output=2)

    # 손실 함수 정의 (예: 크로스 엔트로피)
    criterion = torch.nn.CrossEntropyLoss()

    # 옵티마이저 설정 (예: 확률적 경사 하강법 - SGD)
    optimizer = torch.optim.SGD(my_model.parameters(), lr=0.01)

    # 학습 루프
    num_epochs = 1500  # 에폭 수를 설정
    for epoch in range(num_epochs):
        my_model.train()  # 모델을 학습 모드로 설정
        running_loss = 0.0

        for batch in train_data_loader:
            inputs = batch['input']
            labels = batch['target']

            optimizer.zero_grad()
            outputs = my_model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
        
        # Training loss를 wandb에 기록
        # running_loss를 train_data_loader로 나누는 이유 : 코드에서 배치 사이즈를 16으로 설정했는데 그것을 1개의 loss로 만들어주기 위해서
        wandb.log({"Training Loss": running_loss / len(train_data_loader)})

        print(f"Epoch {epoch + 1}, Training Loss: {running_loss / len(train_data_loader)}")

        # 검증 부분
        my_model.eval()  # 모델을 평가 모드로 설정
        validation_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for batch in validation_data_loader:
                inputs = batch['input']
                labels = batch['target']

                outputs = my_model(inputs)
                loss = criterion(outputs, labels)
                validation_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        
        # Validation loss와 accuracy를 wandb에 기록
        # validation_loss를 validation_data_loader로 나누는 이유 : 코드에서 배치 사이즈를 16으로 설정했는데 그것을 1개의 loss로 만들어주기 위해서
        wandb.log({"Validation Loss": validation_loss / len(validation_data_loader), "Validation Accuracy": accuracy})
        
        print(f"Epoch {epoch + 1}, Validation Loss: {validation_loss / len(validation_data_loader)}, Validation Accuracy: {accuracy:.2f}%")

    print("#" * 50, 3)

    # 테스트 함수 호출
    test(test_data_loader)
    
    wandb.finish()

    # 예측 및 승객 ID를 저장할 빈 리스트를 생성
    predictions = []
    passenger_ids = []

    # 모델을 평가 모드로 설정합니다.
    my_model.eval()

    # 테스트 데이터의 승객 ID 범위 (892부터 1309까지)를 정의
    test_passenger_id_range = range(892, 1310)

    with torch.no_grad():
        for batch in test_data_loader:
            inputs = batch['input']
            passenger_ids += test_passenger_id_range  # 892부터 1309까지의 승객 ID를 추가

            outputs = my_model(inputs)
            _, predicted = torch.max(outputs, 1)
            predictions += predicted.tolist()

    # 제출용 DataFrame을 생성
    submission_df = pd.DataFrame({'PassengerId': passenger_ids, 'Survived': predictions})

    # DataFrame을 CSV 파일로 저장
    submission_df.to_csv('submission.csv', index=False)

train_dataset: 713, validation_dataset.shape: 178, test_dataset: 418
################################################## 1
0 - tensor([  1.0000,   0.0000,  14.0000,   1.0000,   2.0000, 120.0000,   2.0000,
         87.5090,   1.0000,   3.0000,   0.0000]): 1
1 - tensor([ 2.0000,  1.0000, 24.0000,  0.0000,  0.0000, 10.5000,  2.0000, 21.1792,
         2.0000,  0.0000,  1.0000]): 0
2 - tensor([ 3.0000,  1.0000, 27.0000,  0.0000,  0.0000,  6.9750,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 1
3 - tensor([ 3.0000,  1.0000, 29.0000,  0.0000,  0.0000,  7.8958,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
4 - tensor([ 3.0000,  1.0000, 20.0000,  0.0000,  0.0000,  7.8542,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
5 - tensor([ 3.0000,  1.0000, 32.0000,  0.0000,  0.0000,  7.7500,  1.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
6 - tensor([ 3.0000,  1.0000, 27.0000,  0.0000,  0.0000,  8.6625,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 1
7 - te

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)


Epoch 147, Training Loss: 0.48884932266341313
Epoch 147, Validation Loss: 0.46467548112074536, Validation Accuracy: 79.78%
Epoch 148, Training Loss: 0.4882314943604999
Epoch 148, Validation Loss: 0.474320641408364, Validation Accuracy: 76.97%
Epoch 149, Training Loss: 0.48686240050527785
Epoch 149, Validation Loss: 0.46239378784472746, Validation Accuracy: 73.60%
Epoch 150, Training Loss: 0.4827360851897134
Epoch 150, Validation Loss: 0.43686551849047345, Validation Accuracy: 76.40%
Epoch 151, Training Loss: 0.4594980203443103
Epoch 151, Validation Loss: 0.47556421160697937, Validation Accuracy: 74.16%
Epoch 152, Training Loss: 0.4853038390477498
Epoch 152, Validation Loss: 0.47770588224132854, Validation Accuracy: 74.72%
Epoch 153, Training Loss: 0.5055506189664205
Epoch 153, Validation Loss: 0.4734242806831996, Validation Accuracy: 75.84%
Epoch 154, Training Loss: 0.46774267620510523
Epoch 154, Validation Loss: 0.5075073999663194, Validation Accuracy: 75.28%
Epoch 155, Training Loss:

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)


Epoch 750, Training Loss: 0.3879067791832818
Epoch 750, Validation Loss: 0.7370825124283632, Validation Accuracy: 75.28%
Epoch 751, Training Loss: 0.4069558097256554
Epoch 751, Validation Loss: 0.5772180370986462, Validation Accuracy: 71.91%
Epoch 752, Training Loss: 0.3780321094724867
Epoch 752, Validation Loss: 0.43623123814662296, Validation Accuracy: 81.46%
Epoch 753, Training Loss: 0.3792836950884925
Epoch 753, Validation Loss: 0.49071670323610306, Validation Accuracy: 79.21%
Epoch 754, Training Loss: 0.3738553328646554
Epoch 754, Validation Loss: 0.5647425999244055, Validation Accuracy: 79.78%
Epoch 755, Training Loss: 0.3770121836000019
Epoch 755, Validation Loss: 0.4629059446354707, Validation Accuracy: 80.34%
Epoch 756, Training Loss: 0.37453631195757126
Epoch 756, Validation Loss: 0.4999184872334202, Validation Accuracy: 77.53%
Epoch 757, Training Loss: 0.41239718761709
Epoch 757, Validation Loss: 0.5864676584800085, Validation Accuracy: 75.28%
Epoch 758, Training Loss: 0.379

wandb: ERROR Dropped streaming file chunk (see wandb/debug-internal.log)


Training Loss,█▇▇▆▆▅▅▄▄▄▄▃▄▄▃▃▃▃▃▃▃▂▃▃▂▂▂▂▂▂▁▂▂▂▂▁▁▁▁▂
Validation Accuracy,▆▇▇▇█▇█▅██▇▇▅▁▆█▇███▆█▇█▇▇▇▇▇█████▇█████
Validation Loss,▃▂▁▁▂▁▂▃▁▁▁▂▃█▂▂▂▃▁▂▅▂▃▃▂▄▂▂▂▁▁▄▂▂▂▂▂▂▂▂
Training Loss,0.36019
Validation Accuracy,80.33708
Validation Loss,0.54469


# Kaggle 이미지

![](leaderboard.png)

## 기술적 사항 / 고찰

### - 절대 경로를 설정하는 코드인 CURRENT_FILE_PATH = os.path.dirname(os.path.abspath(__file__))는 .py파일 에서는 (__file__)로 돌아가지만 jupyter notebook에서는 (__file__)이 아닌 ("__file__")로 해야 돌아가는 것을 알 수 있었고 오류가 나는 이유는 절대 경로 설정이 .py 파일과 .ipynb가 달라서 안되는 것 같습니다.

### - Titanic_Dataset 에는 PassengerId, Name, Pclass, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked 11개의 feature와 각 데이터들이 들어있고, 이 데이터들을 텐서화 하고 특정 데이터 반환, 데이터셋의 정보를 알려주는 코드를 작성해보면 데이터 처리를 공부 할 수 있었습니다.

### - get_preprocessed_dataset 함수에서는 .csv 파일을 절대 경로 설정해서 train.csv, test.csv 파일을 .concat을 통해 데이터를 합치고 get_preprocessed_dataset_1 부터 6까지의 데이터 전처리 과정을 통해 얻은 결과들을 넣는 함수 입니다. 전처리를 통해 Name과 같은 Mr. Miss. 성, 이름과 같이 분리되어 지는 데이터는 전처리를 통하여 데이터 처리하는 방법을 알 수 있었습니다.

### - nn.Module을 통해 신경망을 정의하고 정의한 신경망을 통해 나온 출력 값을 foward를 통해 feed fowarding하는 방법을 알 수 있었고, Activation Function의 변경을 통해 ReLU, Leaky ReLU, PReLU등 더 나은 성능을 산출하는 Activation Function이 있는지 조사했습니다.

### - main문 에서 학습되어 나온 데이터의 loss를 구하는 Training_loss, Validation_loss를 구했습니다. loss를 구할 때 loss를 train_data_loader로 나누는 이유는 배치를 16으로 설정한 것을 나눠야 loss값을 구할 수 있기 때문입니다.

### - loss를 구한 값을 wandb를 사용하여 wandb.log로 올려보내 Training_loss, Validation_loss, Validation_accuracy를 그래프화 하여 알려주는 방법을 알 수 있었습니다. wandb를 사용하기 위해서는 wandb.init을 사용하여 초기화를 수행하고, wandb.log를 통하여 원하는 값을 올려주고, wandb.finish를 사용하여 끝내는 것을 알 수 있었습니다.

### - 마지막으로 테스트를 통해 나온 생존자(Survived)의 생존 여부 값인 0과 1을 submission.csv파일로 저장하여 PassengerId 892번 부터 ~ 1309번 까지의 생존 여부를 예측한 파일을 만들어 캐글에 제출했습니다.


# 숙제 후기

### - 절대 경로를 설정하는 코드인 CURRENT_FILE_PATH = os.path.dirname(os.path.abspath(__file__))는 .py파일 에서는 (__file__)로 돌아가지만 jupyter notebook에서는 (__file__)이 아닌 ("__file__")로 해야 돌아가는 것을 찾는데 되게 많이 고생했고, wandb를 어떻게 올려 보내는지 몰라 처음에는 많이 고생했지만 결국 어떠한 방법을 사용하는지 알 수 있었고 배울 내용도 되게 많아서 좋았습니다. 다만 데이터 처리하는데 있어서 RuntimeError: The size of tensor a (11) must match the size of tensor b (16) at non-singleton dimension 1 오류가 발생했는데 제가 아직 데이터를 처리하는게 미숙해서 인것 같습니다. 따라서 해당 부분을 공부를 조금더 해야할 것 같습니다.